In [1]:
import gymnasium as gym
import nbformat



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class_path= "/content/drive/MyDrive/husob/06 ReinForcement Learning/004/Drone_env.ipynb"

In [4]:
with open(class_path, 'r') as f:
    nb = nbformat.read(f, as_version=4)

In [5]:
for cell in nb.cells:
    if cell.cell_type == 'code':
       try:
        exec(cell.source, globals())
       except Exception as e:
        print(f"Error executing cell: {e}")


Error executing cell: not enough values to unpack (expected 5, got 4)


/usr/local/lib/python3.11/dist-packages/gymnasium/utils/passive_env_checker.py:212: DeprecationWarning:

WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 



In [6]:
 # الأبنية
buildings = [
      {'pos': [3, 3, 0], 'size': [5, 5, 5]},
      {'pos': [8, 8, 0], 'size': [1, 1, 1]}
  ]
  # المناطق الممنوعة
prohibited_zones = [
      {'pos': [0, 2, 0], 'size': [2, 4, 4]},
      {'pos': [8, 1, 0], 'size': [2, 2, 5]}
  ]
  # البداية
start_position = [1, 9, 0]
  # الهدف
goal_position = [9, 0, 8]

  # إنشاء البيئة
env = gym.make("Drone-v0",
               grid_size=10,
               start_position=start_position,buildings=buildings,
               prohibited_zones=prohibited_zones ,
               max_episode_steps=400)

env.reset()
raw_env = env.unwrapped
raw_env.set_goal(goal_position )



# # الإظهار
# env.render()

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning:

WARN: The environment Drone-v0 is out of date. You should consider upgrading to version `v1`.



In [7]:
import numpy as np
import time
from tqdm import trange
from IPython.display import clear_output

# --- Q-Learning Agent ---
class QLearningAgent:
    """
    A Q-Learning agent for discrete observation and action spaces.
    """
    def __init__(self, observation_space_size, action_space_size, learning_rate=0.1, discount_factor=0.99, epsilon=1.0):
        self.q_table = np.zeros((observation_space_size, action_space_size))
        self.learning_rate = learning_rate  # Alpha
        self.discount_factor = discount_factor  # Gamma
        self.epsilon = epsilon  # Exploration factor

    def choose_action(self, state):
        """
        Chooses an action using an epsilon-greedy policy.
        """
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.q_table.shape[1])  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def learn(self, state, action, reward, next_state):
        """
        Q(s,a) = Q(s,a) + alpha * (reward + gamma * max(Q(s',a')) - Q(s,a))
        """
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state])
        new_value = old_value + self.learning_rate * (reward + self.discount_factor * next_max - old_value)
        self.q_table[state, action] = new_value


# --- Training Function ---
def train_q_learning(env, agent, num_episodes=10000, epsilon_decay_rate=0.999, min_epsilon=0.01,
                     render_each_n_episodes=1000, seed=None):
    """
    Trains a Q-Learning agent in the given environment.
    """
    if seed is not None:
        np.random.seed(seed)
        env.reset(seed=seed)

    rewards_per_episode = []
    best_q_table = None
    best_avg_reward = float('-inf')

    print(f"Starting Q-Learning training for {num_episodes} episodes...")

    for episode in trange(num_episodes):
        state, _ = env.reset()
        done = False
        total_episode_reward = 0

        while not done:
            action = agent.choose_action(state)
            next_state, reward, terminated, truncated, _ = env.step(action)

            agent.learn(state, action, reward, next_state)
            state = next_state
            total_episode_reward += reward
            done = terminated or truncated

        # Update epsilon
        agent.epsilon = max(min_epsilon, agent.epsilon * epsilon_decay_rate)
        rewards_per_episode.append(total_episode_reward)

        # Track best Q-table
        if episode >= 100:
            avg_reward = np.mean(rewards_per_episode[-100:])
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                best_q_table = agent.q_table.copy()
        # clear screen very 100000
        if (episode + 1) % 20000 == 0:
          clear_output(wait=True)
        # Logging
        if (episode + 1) % 1000 == 0:

            avg_reward = np.mean(rewards_per_episode[-100:])
            print(f"Episode {episode + 1}: Avg Reward (last 100) = {avg_reward:.2f}, Epsilon = {agent.epsilon:.4f}")

        # # Rendering
        # if (episode + 1) % render_each_n_episodes == 0:
        #     print(f"\n--- Rendering Episode {episode + 1} ---")
        #     clear_output(wait=True)
        #     env.render()
        #     time.sleep(2)

    print("\nTraining complete!")
    return {
        "rewards": rewards_per_episode,
        "q_table": agent.q_table,
        "best_q_table": best_q_table
    }





In [14]:

agent = QLearningAgent(observation_space_size=env.observation_space.n, action_space_size=env.action_space.n)
ans = train_q_learning(env,
                agent,
                 num_episodes=2_000_000,
                 epsilon_decay_rate=0.999995,
                 min_epsilon=0.01,
                 render_each_n_episodes=50000
                 )



100%|██████████| 2000000/2000000 [20:15<00:00, 1645.12it/s]

Episode 2000000: Avg Reward (last 100) = 975.20, Epsilon = 0.0100

Training complete!


In [15]:
import pickle
def save_q_table_and_env(q_table,best_q_table,env , q_filename,best_q_filename,enV_filename):
    with open(q_filename, 'wb') as f:
        pickle.dump(q_table, f)
    with open(best_q_filename, 'wb') as f:
        pickle.dump(best_q_table, f)
    with open(enV_filename, 'wb') as f:
        pickle.dump(env, f)


In [16]:
working_dir = "/content/drive/MyDrive/husob/06 ReinForcement Learning/004/"
q_table_name = working_dir + "q_table1.pkl"
best_q_table_name = working_dir + "best_q_table1.pkl"
env_name = working_dir + "env1.pkl"
save_q_table_and_env(ans["q_table"],ans["best_q_table"],env,q_table_name,best_q_table_name,env_name)
# save_q_table_and_env(,env,best_q_table_name,env_name)

In [ ]:
#  # الأبنية
# buildings = [
#       {'pos': [3, 3, 0], 'size': [5, 5, 5]},
#       {'pos': [8, 8, 0], 'size': [1, 1, 1]}
#   ]
#   # المناطق الممنوعة
# prohibited_zones = [
#       {'pos': [0, 2, 0], 'size': [2, 4, 4]},
#       {'pos': [8, 1, 0], 'size': [2, 2, 5]}
#   ]
#   # البداية
# start_position = [1, 9, 0]
#   # الهدف
# goal_position = [9, 0, 8]

#   # إنشاء البيئة
# env2 = gym.make("Drone-v1",
#                grid_size=10,
#                start_position=start_position,buildings=buildings,
#                prohibited_zones=prohibited_zones ,
#                max_episode_steps=200)

# env2.reset()
# raw_env2 = env.unwrapped
# raw_env2.set_goal(goal_position )


In [ ]:
# agent2 = QLearningAgent(observation_space_size=env2.observation_space.n, action_space_size=env2.action_space.n)
# ans2 = train_q_learning(
#                 env2,
#                 agent2,
#                  num_episodes=2_000_000,
#                  epsilon_decay_rate=0.999992,
#                  min_epsilon=0.01,
#                 #  render_each_n_episodes=50000
#                  )

In [ ]:
# working_dir = "/content/drive/MyDrive/husob/06 ReinForcement Learning/004/"
# q_table_name = working_dir + "q_table2.pkl"
# env_name = working_dir + "env2.pkl"
# save_q_table_and_env(ans2["q_table"],env2,q_table_name,env_name)